In [2]:
import time
import random

class GameTree:
    def __init__(self, state):
        self.state = state
        self.children = []

    def get_state(self):
        return self.state

    def set_value(self, value):
        self.value = value
    
    def get_value(self):
        return self.value
    
    def add_child(self, child_state):
        self.children.append(child_state)
    
    def get_children(self):
        return self.children

In [3]:
def grid(rows, cols):
    poison_row = rows - 1
    poison_col = 0
    return [['P' if (c == poison_col) and (r == poison_row) else 'C' for c in range(cols)] for r in range(rows)], poison_row, poison_col

# def grid(rows, cols):
#    poison_row = random.randint(0, rows - 1)
#    poison_col = random.randint(0, cols - 1)
#    return [['P' if (c == poison_col) and (r == poison_row) else 'C' for c in range(cols)] for r in range(rows)], poison_row, poison_col

def boundary(cols):
    for c in range(cols):
        print('--', end='')
    print('')

def display(grid, rows, cols):   
    boundary(cols)   
    for r in range(rows):
        for c in range(cols):
            print(grid[r][c] + ' ', end='')
        print('')

In [4]:
def get_user_input(rows, cols):
    user_row = int(input("Select row: "))
    user_col = int(input("Select column: "))

    while user_row >= rows or user_col >= cols or user_row < 0 or user_col < 0:
        boundary(cols)
        print("Invalid location, try again!")
        user_row = int(input("Select row: "))
        user_col = int(input("Select column: "))
    return user_row, user_col

def get_new_state(state, user_row, user_col, rows, cols):
    new_state = [[state[r][c] for c in range(cols)] for r in range(rows)]
    for r in range(user_row + 1):
        for c in range(user_col, cols):
            if state[r][c] != '-':
                new_state[r][c] = '-'
    return new_state

In [29]:
def generate_children(parent_state, rows, cols, depth, poison_row, poison_col):
    if depth < 5:
        current_state = parent_state.get_state()

        if current_state[poison_row][poison_col] != '-':
            for r in range(rows):
                if len(parent_state.get_children()) < 11:
                    for c in range(cols):
                        if current_state[r][c] == '-':
                            continue
                        else:
                            if len(parent_state.get_children()) < 11:
                                new_state = get_new_state(current_state, r, c, rows, cols)
                                child_state = GameTree(new_state)
                                parent_state.add_child(child_state)
                else:
                    break

            for child in parent_state.get_children():
                generate_children(child, rows, cols, depth + 1, poison_row, poison_col)
        
        else:
            if depth % 2 == 1:
                parent_state.set_value(depth - (rows * cols))
            else:
                parent_state.set_value(depth)

In [30]:
def minimax(parent_state, depth):
    if depth < 5:
        if parent_state.get_children() == []:
            return parent_state.get_state(), parent_state.get_value()
        
        else:
            if depth % 2 == 0:
                parent_state.set_value(float("-inf"))
            else:
                parent_state.set_value(float("inf"))

            for child in parent_state.get_children():
                _, value = minimax(child, depth + 1)
                if depth % 2 == 0:
                    if value >= parent_state.get_value():
                        parent_state.set_value(value)
                        return_state = child.get_state()
                else:
                    if value <= parent_state.get_value():
                        parent_state.set_value(value)
                        return_state = child.get_state()
            
            return return_state, value
    
    else:
        if depth % 2 == 0:
            value = float("-inf")
        else:
            value = float("inf")
        return parent_state.get_state(), value

In [34]:
rows = int(input("Grid rows: "))
cols = int(input("Grid columns: "))
game_state, poison_row, poison_col = grid(rows, cols)
display(game_state, rows, cols)

while True:
    user_row, user_col = get_user_input(rows, cols)
    while game_state[user_row][user_col] == '-':
        boundary(cols)
        print("Empty location, try again!")
        user_row, user_col = get_user_input(rows, cols)

    game_state = get_new_state(game_state, user_row, user_col, rows, cols)
    display(game_state, rows, cols)

    if game_state[poison_row][poison_col] == '-':
        boundary(cols)
        print("YOU LOSE!")
        break

    parent_state = GameTree(game_state)
    generate_children(parent_state, rows, cols, 0, poison_row, poison_col)
    game_state, _ = minimax(parent_state, 0)
    time.sleep(1)
    display(game_state, rows, cols)

    if game_state[poison_row][poison_col] == '-':
        boundary(cols)
        print("YOU WIN!")
        break

----------
C C C C C 
C C C C C 
C C C C C 
C C C C C 
P C C C C 
----------
- - - - - 
- - - - - 
- - - - - 
- - - - - 
P C C C C 
----------
- - - - - 
- - - - - 
- - - - - 
- - - - - 
P C C C - 
----------
- - - - - 
- - - - - 
- - - - - 
- - - - - 
P C - - - 
----------
- - - - - 
- - - - - 
- - - - - 
- - - - - 
P - - - - 
----------
- - - - - 
- - - - - 
- - - - - 
- - - - - 
- - - - - 
----------
YOU LOSE!
